In [8]:
# Get data here: https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/
import pandas as pd
import numpy as np
from rgf.sklearn import RGFRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("Big_Mart_Train.csv")
test = pd.read_csv("Big_Mart_Test.csv")

In [3]:
train.Item_Weight = train.Item_Weight.fillna(np.nanmedian(train.Item_Weight))
test.Item_Weight = test.Item_Weight.fillna(np.nanmedian(test.Item_Weight))

In [4]:
train.Outlet_Size = train.Outlet_Size.fillna(train.Outlet_Size.mode().iloc[0])
test.Outlet_Size = test.Outlet_Size.fillna(train.Outlet_Size.mode().iloc[0])

In [5]:
train.Item_Fat_Content = train.Item_Fat_Content.replace(['low fat', 'LF'], ['Low Fat', 'Low Fat'])
test.Item_Fat_Content = test.Item_Fat_Content.replace(['low fat', 'LF'], ['Low Fat', 'Low Fat'])
train.Item_Fat_Content = train.Item_Fat_Content.replace(['reg'], ['Regular'])
test.Item_Fat_Content = test.Item_Fat_Content.replace(['reg'], ['Regular'])

In [6]:
from sklearn.model_selection import train_test_split

train = train.drop('Item_Identifier',axis=1)
test = test.drop('Item_Identifier',axis=1)

y = train.Item_Outlet_Sales
X = train.drop(['Item_Outlet_Sales'], axis=1)

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=410)

In [10]:
parameters = {'max_leaf':[1000,1200,1300,1400,1500,1600,1700,1800,1900,2000],
              'l2':[0.1,0.2,0.3],
              'min_samples_leaf':[4,10]}
clf = GridSearchCV(estimator='rgf',
                   param_grid=parameters,
                   scoring='neg_mean_squared_error',
                   n_jobs = -1,
                   cv = 3)